In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

Cloning into 'yolov7'...
remote: Enumerating objects: 1157, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1157 (delta 10), reused 15 (delta 8), pack-reused 1139
Receiving objects: 100% (1157/1157), 70.41 MiB | 35.12 MiB/s, done.
Resolving deltas: 100% (498/498), done.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00


In [ ]:
!pip install gradio
!pip install easyocr
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
import gradio as gr
import cv2
from PIL import ImageDraw, ImageFont
import numpy as np
from torchvision.transforms import functional as F
import torch
from PIL import Image
import easyocr
import openai

In [ ]:
%pwd

'/content/yolov7'

In [12]:
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = [res[1] for res in reader.readtext('/content/drive/MyDrive/yogurt.jpg')]
result_single_line = " ".join(result).replace('\n', ' ').strip()
print(result_single_line)

60 CALORIES LACTOSE Chobani Zero Sugar Blueberry FLAVOR NATURAL INGREDIENTs ONLY


In [13]:
template = """
I want you to act as a nutrition dietitian.
What is the benefit of eating this {product}?
"""
llm = OpenAI(openai_api_key="")
output_parser = CommaSeparatedListOutputParser()
prompt = PromptTemplate(
            input_variables=["product"],
            template=template,
            output_parser=output_parser
)
prompt.format(product=result_single_line)
chain = LLMChain(llm=llm, prompt=prompt)
result_langchain = chain.predict_and_parse(product=result_single_line)
print("\n".join(result_langchain))

The benefit of eating this 60 calories Chobani Zero Sugar Blueberry FLAVOR NATURAL INGREDIENTs ONLY is that it is a great source of nutrition without added sugar. This product is free of artificial ingredients and sweeteners
providing a source of essential vitamins and minerals. It is also low in calories
making it a great snack choice for those looking to maintain a healthy weight. Additionally
it is a good source of probiotics
which can help to support overall digestive health.


In [14]:
import easyocr
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser



template = """
I want you to act as a nutrition dietitian.
What is the benefit of eating this {product}?
"""

# Load the YOLOv7 model
model = torch.hub.load('.', 'custom', '../drive/MyDrive/best_weights.pt', source='local')
model.eval()

def process_image(input_image, item):
    try:
        # Convert Gradio input image (numpy array) to PIL Image
        flipped_image = np.fliplr(input_image)

        processed_image = Image.fromarray(flipped_image.astype('uint8'), 'RGB')

        # Perform inference
        results = model(processed_image)

        detected_objects = results.pandas().xyxy[0]
        if item in detected_objects["name"].values:
            result = "Yes! the Item you are looking for is found"
        else:
            result = "No! the Item you are looking for is not found"

        
        # Get the annotated image from the results
        annotated_image = results.render()[0]
        
        # Convert the annotated image to numpy array for Gradio
        annotated_image = annotated_image[:, :, ::-1]  # Convert RGB to BGR
        
        return Image.fromarray(annotated_image.astype('uint8')), result

    except Exception as e:
        print(f"Error: {str(e)}")

def process_text(input_image):
    try:
        flipped_image = np.fliplr(input_image)
        reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
        result = [res[1] for res in reader.readtext(flipped_image)]
        result_single_line = " ".join(result).replace('\n', ' ').strip()
        llm = OpenAI(openai_api_key="")
        output_parser = CommaSeparatedListOutputParser()
        prompt = PromptTemplate(
            input_variables=["product"],
            template=template,
            output_parser=output_parser
        )
        prompt.format(product=result_single_line)
        chain = LLMChain(llm=llm, prompt=prompt)
        result_langchain = chain.predict_and_parse(product=result_single_line)
        return "\n".join(result_langchain) # Extract only the text from the result
    except Exception as e:
        print(f"Error: {str(e)}")

def process_image_and_text(input_image, item):
    processed_image, image_result = process_image(input_image.copy(), item)  # Call your image processing function with a copy of the input image
    text_output = process_text(input_image)  # Call your text generation function with the original input image
    return  processed_image, image_result, text_output


iface = gr.Interface(
    fn=process_image_and_text,
    inputs=[
        gr.inputs.Image(source="webcam", tool="opencv", type="numpy"),
        gr.inputs.Textbox(label="Enter the item you are looking for")
    ],
    # outputs=[
        # gr.outputs.Image(type="pil", label="Processed Image"),
        # gr.outputs.Textbox(label="Text Output")
    # ],
    outputs=["image", "text", "text"],
    title="Know Your Grocery!! powered by YOLOv7",
    description="Click a photo and Detect grocery products in the image using YOLOv7.",
    allow_flagging=False,
    theme="default",
    layout="vertical",
    server_port=7860
)

iface.launch(inbrowser=True)


Adding autoShape... 


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
<ipython-input-14-b48685e689a3>:72: UserWarning: `layout` parameter is deprecated, and it has no effec

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>